<a href="https://colab.research.google.com/github/jjung2-oxy/Natural-Lang-Processing/blob/main/Soccer_Embeddings_(NLP_Final_Project).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install statsbombpy
!pip install umap-learn
!pip install chart_studio
import statsbombpy
from statsbombpy import sb
import pandas as pd
from tqdm import tqdm as tqdm
import os
import json
import ast 
import numpy as np
import random
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import KeyedVectors
import plotly.express as px
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import umap
import chart_studio
import chart_studio.plotly as py

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
# LOADS ALL THE NECESSARY DATA FROM THE DATASET

def load_events():
    data_raw = []
    directory = r'/content/drive/MyDrive/data/sample_events'
    files = os.listdir(directory)

    print('\nLOADING ALL EVENTS')
    for match in tqdm(files, total=len(files)):
        with open(f'/content/drive/MyDrive/data/sample_events/{match}') as data_file:
            data = json.load(data_file)
            data_raw.append(pd.json_normalize(data, sep='_').assign(match_id=match))

    all_events = pd.DataFrame()
    all_events = all_events.append(data_raw, ignore_index=True)
    return all_events

def load_player_metadata():
    data = []
    dir = r'/content/drive/MyDrive/data/lineups'
    files = os.listdir(dir)

    print('\nLoading Player Metadata')
    for file_name in tqdm(files, total=len(files)):
        with open(f'/content/drive/MyDrive/data/lineups/{file_name}') as data_file:
            data_item = json.load(data_file)
            home_line_up, away_line_up = data_item[0], data_item[1]
            for player in home_line_up['lineup']:
                data.append(pd.json_normalize(player, sep='_'))
            for player in away_line_up['lineup']:
                data.append(pd.json_normalize(player, sep='_'))

    all_players_metadata = pd.DataFrame()
    all_players_metadata = all_players_metadata.append(data, ignore_index=True)
    return all_players_metadata

def get_team_metadata():

    data = []
    dir = r'/content/drive/MyDrive/data/matches'
    competitions_seasons = [name for name in os.listdir(dir)]
    print('\nLoading Team Metadata')

    for season in tqdm(competitions_seasons, total=len(competitions_seasons)):
        files = os.listdir(f'/content/drive/MyDrive/data/matches/{season}')
        for file_name in files:
            with open(f'/content/drive/MyDrive/data/matches/{season}/{file_name}') as data_file:
                data_item = json.load(data_file)
                for item in data_item:
                    data.append(pd.json_normalize(item))

    all_teams_metadata = pd.DataFrame()
    all_teams_metadata = all_teams_metadata.append(data, ignore_index=True)

    # Columns that apply to both home and away
    neutral_cols = ['match_id', 'season.season_name', 'stadium.name', 'competition.competition_name', 
            'competition.country_name', 'competition_stage.name']

    # Home Team Data
    home_teams_metadata = all_teams_metadata[neutral_cols + ['home_team.home_team_name', 'home_team.country.name', 'home_team.home_team_gender']]
    # Away Team Data
    away_teams_metadata = all_teams_metadata[neutral_cols + ['away_team.away_team_name', 'away_team.country.name', 'away_team.away_team_gender']]
    
    # Rename shared columns
    cols_mapper = {'season.season_name': 'season_name', 'competition.country_name': 'competition_country_name', 'competition_stage.name': 'stage_name', 'competition.competition_name': 'competition_name'}
    home_teams_metadata.rename(columns=cols_mapper, inplace=True)
    away_teams_metadata.rename(columns=cols_mapper, inplace=True)
    # Rename home/away team columns seperately 
    home_teams_metadata.rename(columns={'home_team.home_team_name': 'team_name',
                                        'home_team.home_team_gender': 'team_gender',
                                        'home_team_managers': 'team_managers',
                                        'home_team.country.name': 'country_name'}, inplace=True)
    print(f"Length of home_teams_metadata: {len(home_teams_metadata)}")
    away_teams_metadata.rename(columns={'away_team.away_team_name': 'team_name',
                                        'away_team.away_team_gender': 'team_gender',
                                        'away_team_managers': 'team_managers',
                                        'away_team_country_name': 'country_name'}, inplace=True)
    print(f"Length of away_teams_metadata: {len(away_teams_metadata)}")

    # Append home and away metadata together (each row is a match metadata of one team)
    teams_metadata = pd.DataFrame()
    teams_metadata = teams_metadata.append(home_teams_metadata, ignore_index=True)
    teams_metadata = teams_metadata.append(away_teams_metadata, ignore_index=True)
    teams_metadata.drop('away_team.country.name', axis=1, inplace=True)
    print(f"Length of combined teams_metadata: {len(teams_metadata)}")

    return teams_metadata


def load_matches_metadata() -> pd.DataFrame:
    data = []

    print('\nLoading Matches Metadata')
    dir = f'/content/drive/MyDrive/data/matches'
    competitions_dirs = os.listdir(dir)
    for season in tqdm(competitions_dirs, total=len(competitions_dirs)):
        try:
            files = os.listdir(f'/content/drive/MyDrive/data/matches/{season}')
        except NotADirectoryError:
            continue
        for file_name in files:
            with open(f'/content/drive/MyDrive/data/matches/{season}/{file_name}') as data_file:
                data_item = json.load(data_file)
                for item in data_item:
                    data.append(pd.json_normalize(item, sep="_"))

    matches_metadata = pd.DataFrame()
    matches_metadata = matches_metadata.append(data, ignore_index=True)
    cols_mapper = {'season_season_name': 'season_name', 'competition_competition_name': 'competition_name'}
    matches_metadata.rename(columns=cols_mapper, inplace=True)
    matches_metadata = matches_metadata.drop_duplicates(subset=['match_id'])
    
    return matches_metadata

all_events = load_events()

In [ ]:
# FUNCTIONS FOR ADDITIONAL ACTION ATTRIBUTES

def get_centered_coordinates(data):

    field_dimensions = (106.0, 68)

    # Get all x coordinates and all y coordinates
    x_columns = [c for c in data.columns if (c[-2:].lower() == 'x')]
    y_columns = [c for c in data.columns if c[-2:].lower() == 'y']

    # Convert yard to meters
    for x_col in x_columns:
        data[x_col] = (0.9144 * data[x_col]) - 0.5 * field_dimensions[0]
    for y_col in y_columns:
        data[y_col] = (-1 * (0.9144 * data[y_col]) - 0.5 * field_dimensions[1])

    for col in [col for col in data.columns if 'location' in col]:
        data[col] = data[col].apply(lambda val: ast.literal_eval(val) if isinstance(val, str) else val)
        data[col] = data[col].apply(lambda val: ((0.9144 * val[0]) - 0.5 * field_dimensions[0]), (-1 * (0.9144 * val[1]) - 0.5 * field_dimensions[1]))

    return data

def yard_meter_convert(yards):
    return 0.9144 * yards


def get_centered_coordinates(data):
    '''
    Convert positions units to meters with origin at centre circle
    '''

    field_length = 106.0
    field_width = 68

    x_columns = [c for c in data.columns if (c[-2:].lower() == '_x')]
    y_columns = [c for c in data.columns if c[-2:].lower() == '_y']
    for x_col in x_columns:
        data[x_col] = (yard_meter_convert(data[x_col]) - 0.5 * field_length)
    for y_col in y_columns:
        data[y_col] = -1 * (yard_meter_convert(data[y_col]) - 0.5 * field_width)

    for col in [col for col in data.columns if 'location' in col]:
        data[col] = data[col].apply(lambda string: ast.literal_eval(string) if isinstance(string, str) else string)
        data[col] = data[col].apply(lambda val: (yard_meter_convert(val[0]) - 0.5 * field_length, -1 * (yard_meter_convert(val[1]) - 0.5 * field_width)) if isinstance(val, tuple) or isinstance(val, list) else val)
    return data

def check_if_shot_scored(action: pd.Series):
    # Check if shot lead to goal

    # Check if type_name is action
    if 'type_name' in action:
        #Check if action is a shot
        if action['type_name'].lower() == 'shot':
            outcome = action.get('shot_outcome_name', np.nan)

            # Check if shot outcome is 'goal', if yes return true, else return false
            if outcome is not np.nan and outcome.lower() == 'goal':
                return True
            else:
                return False
    return np.nan

def check_if_shot_outside_box(action: pd.Series):

    pitch_dimensions = (106.0, 68)
  
    # Return nan if not a shot
    if action['type_name'] != 'Shot':
        return np.nan

    # Return nan if location is empty
    if pd.isna(action['location']):
        return np.nan

    # Get x, y coordinates from location
    if isinstance(action['location'], str):
        x, y = ast.literal_eval(action['location'])
    elif isinstance(action['location'], list) or isinstance(action['location'], tuple):
        x, y = action['location']

    # Inside box if y is within +- 16.5 from y center and distance from x to x end of pitch < 16.5 or if x distance from goal is more than 16.5 meters
    if x > (pitch_dimensions[0] / 2) - 16.5 and abs(y) < 16.5:
        return False
    else:
        return True

def check_if_dribble_won(action: pd.Series):

    # Check if action is dribble and whether dribble was succesful or unsuccesful
    if 'type_name' in action:
        if action['type_name'].lower() == 'dribble':
            outcome = action.get('dribble_outcome_name', np.nan)
            if outcome is not np.nan and outcome.lower() == 'complete':
                return True
            elif outcome is not np.nan and outcome.lower() == 'incomplete':
                return False
        
    return np.nan

In [ ]:
# Gets the extra attributes for actions 

def get_action_attributes(all_events, verbose=False) -> pd.DataFrame:

    verboseprint = print if verbose else lambda *a, **k: None

    verboseprint('Adding attributes to actions...')

    # Convert team names and player names to lower case
    verboseprint("Converting team/player names to lowercase.")
    for col in ['team_name', 'player_name']:
        all_events[col] = all_events[col].progress_apply(lambda name: name.lower() if isinstance(name, str) else name)

    verboseprint('Adding centered location in meters.')
    all_events = get_centered_coordinates(all_events)
    verboseprint("X Coordinates:")
    all_events['POS_START_X'] = all_events['location'].progress_apply(
        lambda val: val[0] if isinstance(val, tuple) or isinstance(val, list) else val)
    verboseprint("Y Coordinates:")
    all_events['POS_START_Y'] = all_events['location'].progress_apply(
        lambda val: val[1] if isinstance(val, tuple) or isinstance(val, list) else val)

    verboseprint('Adding shot attributes.')
    verboseprint('Ensure action is shot.')
    all_events['is_shot'] = all_events.progress_apply(lambda action: 1 if action['type_name'] == 'Shot' else 0, axis=1)
    verboseprint('Check if shot is goal.')
    all_events['goal'] = all_events.progress_apply(lambda event: float(check_if_shot_scored(event)), axis=1)
    verboseprint('Check if shot is out of box.')
    all_events['out_of_box_shot'] = all_events.progress_apply(lambda event: float(check_if_shot_outside_box(event)), axis=1)
    verboseprint('Check if shot is header.')
    all_events['header_shot'] = all_events.progress_apply(lambda event: 1 if pd.notna(event['shot_body_part_name']) and event['shot_body_part_name'].lower() == 'head' else 0, axis=1)
    verboseprint('Check if shot is penalty. ')
    all_events['penalty_shot'] = all_events.progress_apply(lambda event: 1 if pd.notna(event['shot_type_name']) and event['shot_type_name'].lower() == 'penalty' else 0, axis=1)
    verboseprint('Check if shot is free kick.')
    all_events['free_kick_shot'] = all_events.progress_apply(lambda event: 1 if pd.notna(event['shot_type_name']) and event['shot_type_name'].lower() == 'free kick' else 0, axis=1)


    verboseprint('Adding passing attributes.')
    verboseprint('Get expected assists.')
    all_events['xA'] = all_events['pass_assisted_shot_id'].progress_apply(lambda shot_id: all_events.loc[all_events['id'] == shot_id, 'shot_statsbomb_xg'].iloc[0] if isinstance(shot_id, str) else np.nan)
    pass_recipient = 'pass_recipient_name'
    verboseprint('Get recipient of pass.')
    all_events[pass_recipient] = all_events[pass_recipient].progress_apply(lambda val: val.lower() if isinstance(val, str) else val)

    verboseprint("Adding dribbling attributes.")
    verboseprint('Check if dribble succesful.')
    all_events['dribble_won'] = all_events.progress_apply(lambda action: check_if_dribble_won(action), axis=1)

    return all_events

# Get the extra metadata from the dataset and add it to player Dataframe

def get_enriched_players_metadata(events_data) -> dict:

    players_metadata = load_player_metadata()

    players_metadata['player_name_lower'] = players_metadata['player_name'].apply(lambda val: val.lower() if isinstance(val, str) else val)

    vocab_data_cp = events_data[events_data['player_name'].notna()].copy()
    players_2_positions = vocab_data_cp[
        ['player_name', 'position_name', 'match_id']].copy() \
        .drop_duplicates(). \
        groupby(['player_name', 'position_name']). \
        agg({'match_id': np.size}).reset_index().sort_values(by='match_id', ascending=False)
    players_2_positions = players_2_positions.drop_duplicates(subset=['player_name'], keep='first')
    players_2_positions.set_index('player_name', inplace=True)
    players_2_positions = players_2_positions.to_dict(orient='index')

    players_2_jersey_num = players_metadata[['player_name', 'jersey_number']].copy(). \
        groupby(['player_name']). \
        agg({'jersey_number': np.size}).reset_index().sort_values(by='jersey_number', ascending=False)
    players_2_jersey_num.set_index('player_name', inplace=True)
    players_2_jersey_num = players_2_jersey_num.to_dict(orient='index')

    matches_metadata = load_matches_metadata()
    matches_metadata['match_id'] = matches_metadata['match_id'].astype(str)
    matches_metadata = matches_metadata.set_index('match_id')
    matches_metadata = matches_metadata.to_dict(orient='index')
    vocab_data_cp['match_date'] = vocab_data_cp['match_id'].apply(
        lambda match_: matches_metadata.get(str(match_).split('.')[0], {'match_date': 'unknown'})['match_date'])
    players_2_teams = vocab_data_cp.sort_values(by=['match_date'], ascending=False)[
        ['player_name', 'team_name']].copy()
    players_2_teams = players_2_teams.drop_duplicates(subset=['player_name'], keep='first')
    players_2_teams = players_2_teams.set_index('player_name')
    players_2_teams = players_2_teams.to_dict(orient='index')

    # Add most frequent position name into metadata > position_name
    players_metadata['position_name'] = players_metadata['player_name'].apply( \
        lambda name_: players_2_positions.get(name_.lower(), {'position_name': np.nan})['position_name'])
    players_metadata['jersey_number'] = players_metadata['player_name'].apply( \
        lambda name_: players_2_jersey_num.get(name_, {'jersey_number': np.nan})['jersey_number'])
    players_metadata['team_name'] = players_metadata['player_name'].apply( \
        lambda name_: players_2_teams.get(name_.lower(), {'team_name': ''})['team_name'])
    players_metadata = players_metadata.drop_duplicates(subset=['player_name'], keep='first')
    players_metadata = players_metadata.set_index('player_name')
    players_metadata = players_metadata.to_dict(orient='index')


    return players_metadata

In [ ]:
# Run get_action_attributes and get_enriched_players_metadata functions to enrich action and player data.

all_granular_events = get_action_attributes(all_events, verbose=True)
enriched_player_metadata = get_enriched_players_metadata(all_granular_events)

In [ ]:
# Take x,y location on pitch and convert to bin

def get_location_on_pitch(x, y) -> (int, int):

    pitch_dimensions= (106.0, 68)
    num_x_bins = 6
    num_y_bins = 4

    bin_x_width, bin_y_width = np.ceil(pitch_dimensions[0] / num_x_bins), np.ceil(pitch_dimensions[1] / num_y_bins)
    x, y = x + pitch_dimensions[0] / 2, y + pitch_dimensions[1] / 2

    # Extract bin values [0, num bins - 1]
    bin_x = int(min(np.floor(x / bin_x_width), num_x_bins - 1))
    bin_y = int(min(np.floor(y / bin_y_width), num_y_bins - 1))

    return f"({str(bin_x + 1)}/{str(num_x_bins)}, {str(bin_y + 1)}/{str(num_y_bins)})"

# Tokenize action with action name and location
def action_to_token(action) -> str:

        # Add Action Name
        action_name = action.get('type', action.get('type_name', np.nan))
        
        if action_name is np.nan:
            return np.nan
        else:
            action_name = action_name.lower()

        action_token = f'<{action_name}>'

        # Add Action Location
        if action['location'] is not np.nan:
           
            if isinstance(action['location'], list) or isinstance(action['location'], tuple):
                # Location is list or tuple
                x, y = action['location']
            else:
                # Location is string
                x, y = ast.literal_eval(action['location'])

            location = get_location_on_pitch(x, y)
            action_token = action_token + f"{location}".replace(" ", "")

        return action_token.replace(" ", "_")

# Create corpus from actions.
def actions_to_corpus(all_events) -> pd.DataFrame:

        print(f"vocabulary_data size: {all_events.shape}\n")
        vocabulary_data = all_events.copy()

        # Tokenize event data into strings (uses action_to_token())
        print('\nTokenizing Events...')
        vocabulary_data['token'] = vocabulary_data.progress_apply(lambda action: action_to_token(action), axis=1)
        all_events['token'] = vocabulary_data['token'].copy()
        vocabulary_data = vocabulary_data[~vocabulary_data['token'].isna()]
        print(f"vocabulary_data size after processing and removing empty tokens: {all_events.shape}\n")

        # Create unique vocabulary
        vocabulary = [val for val in vocabulary_data['token'].unique() if val is not np.nan]
        vocabulary.extend(['oov'])  
        print(f'Length of vocabulary (excluding oov):', len(vocabulary) - 1)

        # Create token_to_index and index_to_token
        index_to_token = dict(enumerate(vocabulary))
        index_to_token = {str(key): val for key, val in index_to_token.items()}
        token_to_index = {val: key for key, val in index_to_token.items()}

        # Set token index for each action
        vocabulary_data['token_index'] = vocabulary_data['token'].apply(lambda token: token_to_index.get(token, token_to_index['oov']))
        print(vocabulary_data.head(5))

        # Remove non-relevant columns
        vocabulary_data = vocabulary_data[['token_index', 'token'] + ['match_id', 'period', 'possession']]

        for col in ['match_id', 'period', 'possession']:
            vocabulary_data[col] = vocabulary_data[col].astype(str)
        separator = '-'
        vocabulary_data['sentence_key'] = vocabulary_data[['match_id', 'period', 'possession']].apply(lambda vec: separator.join(vec), axis=1)


        print('\nBuilding sentences...')
      
        # Sentences come from vocabulary_data grouped based on keys
        sentences = vocabulary_data[['sentence_key', 'token_index']].groupby('sentence_key')

        # Sentences are turned into lists, and we add numbered index
        sentences = sentences['token_index'].agg(list).reset_index()
        documents = sentences['sentence_key'].tolist()
        sentences = sentences['token_index'].tolist()

        # Merge sentences smaller than sampling window together
        print('\nConcatenating Documents to build sampling_window sized documents...')
        sampling_window = 5
        corpus = []
        temp_actions_length = 0
        temp_actions = []

        for sentence in sentences:
            if len(sentence) >= sampling_window:
                corpus.append(sentence[:])
            else:
                # Store too small sentence and move on
                temp_actions.extend(sentence[:])
                temp_actions_length += len(sentence)

                # Once actions have built up to sufficient size, add as one sentence. Clear temp actions.
                if temp_actions_length >= sampling_window:
                    corpus.append(temp_actions[:])
                    temp_actions_length = 0
                    temp_actions = []

        print('Final number of sentences:', len(corpus))

        # Update vocabulary
        corpus_flat = set([subitem for item in corpus for subitem in item if type(item) is list])
        vocabulary_index = set([token_to_index[token] for token in vocabulary])

        # Update vocabulary after merging sentences
        vocabulary_index = corpus_flat.intersection(vocabulary_index)
        vocabulary = [index_to_token[token_index] for token_index in vocabulary_index]
        print('Final length of vocabulary:', len(vocabulary))

        return vocabulary, vocabulary_index, sentences, corpus, all_events, token_to_index, index_to_token

In [ ]:
# Train word2vec model over action corpus
def train_Word2Vec(corpus, vocabulary_index, index_to_token, verbose=False):
    
    print('Training Word2Vec model...')
    w2vmodel = Word2Vec(corpus, min_count=10, size=32, window=5)
    w2vmodel.train(corpus, total_examples=len(corpus), epochs=5, compute_loss=True)
    
    # Store just the words + their trained embeddings.
    word_vectors = w2vmodel.wv

    # Create W2V indexed vocabulary based only on chosen vocabulary
    w2v_vocabulary = list(word_vectors.vocab.keys())
    w2v_select_vocab_index = [token for token in w2v_vocabulary if token in vocabulary_index]
    w2v_select_vocab = [index_to_token[token] for token in w2v_select_vocab_index]

    return w2vmodel, word_vectors, w2v_select_vocab, w2v_select_vocab_index

In [ ]:
# Cut down amount of action types in dataframe, keep only 'important' actions.
action_types = ['Ball Receipt*', 'Pressure', 'Dispossessed', 'Duel', 'Ball Recovery', 'Dribbled Past', 'Dribble', 'Interception', 'Block', 'Foul Committed', 'Foul Won', 'Goal Keeper', 'Miscontrol', 'Clearance', 'Offside', 'Pass', 'Shot', 'Carry', 'Goal Keeper']
chosen_granular_events = all_granular_events[all_granular_events['type_name'].isin(action_types)]
chosen_granular_events.tail(5)

In [ ]:
# Run actions_to_corpus to convert action data to corpus

vocabulary, vocabulary_index, sentences, events_corpus, all_events, token_to_index, index_to_token = actions_to_corpus(chosen_granular_events)
all_events['token_index'] = all_events['token'].apply(lambda token: token_to_index.get(token, token_to_index['oov']))

In [ ]:
# Train word2vec model on action corpus

w2vmodel, word_vectors, w2v_limited_vocab, w2v_limited_vocab_index = train_Word2Vec(events_corpus, vocabulary_index, index_to_token, verbose=True)

In [ ]:
# Get cosine similarity of actions individually.
def get_similar_actions(word_vectors, w2v_limited_vocab_index, index_to_token, num_similar=5):

    selected_tokens = random.sample(w2v_limited_vocab_index, num_similar)
    for token_index in selected_tokens:
        print(f'\nToken index: {token_index}')
        token = index_to_token[token_index]
        print(f"Token = {token}")
        similar = word_vectors.most_similar(token_index, topn=num_similar)  # get other similar words
        print(f" - {num_similar} most similar tokens: {[index_to_token[sim[0]] for sim in similar]}")

        similar_neg = word_vectors.most_similar(negative=[token_index])
        print(f" - {num_similar} most similar tokens to NEGATIVE form: " f"{[index_to_token[sim[0]] for sim in similar_neg]}")

        embedded_vocab = pd.DataFrame.from_dict({index_to_token[token_index]: word_vectors[token_index] for token_index in w2v_limited_vocab_index}, orient='index')

    return embedded_vocab

In [ ]:
# Test validity of Word2Vec model by generating action comparisons

embedded_vocab = get_similar_actions(word_vectors, w2v_limited_vocab_index, index_to_token, num_similar=5)

In [ ]:
# Create corpus from player actions 
def players_to_corpus(all_events):
    
        print(f"vocabulary_data size: {all_events.shape}\n")
        vocabulary_data = all_events.copy()

        # Tokenize event data into strings (uses action_to_token())
        print('\nTokenizing Events...')
        vocabulary_data['token'] = vocabulary_data.progress_apply(lambda action: action_to_token(action), axis=1)
        all_events['token'] = vocabulary_data['token'].copy()
        vocabulary_data = vocabulary_data[~vocabulary_data['token'].isna()]
        print(f"vocabulary_data size after processing and removing empty tokens: {all_events.shape}\n")

        # Create unique vocabulary
        vocabulary = [val for val in vocabulary_data['token'].unique() if val is not np.nan]
        vocabulary.extend(['oov'])  
        print(f'Length of vocabulary (excluding oov):', len(vocabulary) - 1)

        # Create token_to_index and index_to_token
        index_to_token = dict(enumerate(vocabulary))
        index_to_token = {str(key): val for key, val in index_to_token.items()}
        token_to_index = {val: key for key, val in index_to_token.items()}

        # Set token index for each action
        vocabulary_data['token_index'] = vocabulary_data['token'].apply(lambda token: token_to_index.get(token, token_to_index['oov']))

        # Remove non-relevant columns
        vocabulary_data = vocabulary_data[['token_index', 'token'] + ['player_name', 'match_id']]

        for col in ['player_name', 'match_id']:
            vocabulary_data[col] = vocabulary_data[col].astype(str)
        separator = '/'
        vocabulary_data['sentence_key'] = vocabulary_data[['player_name', 'match_id']].apply(lambda vec: separator.join(vec), axis=1)


        print('\nBuilding sentences...')
      
        # Sentences come from vocabulary_data grouped based on keys
        sentences = vocabulary_data[['sentence_key', 'token_index']].groupby('sentence_key')

        # Sentences are turned into lists, and we add numbered index
        sentences = sentences['token_index'].agg(list).reset_index()
        documents = sentences['sentence_key'].tolist()
        sentences = sentences['token_index'].tolist()

        # add to corpus sentences that are longer than sampling window
        sampling_window = 5
        document_names = []
        corpus= []
        print('\nBuilding Documents...')
        for i, doc in tqdm(enumerate(sentences)):
            if len(doc) >= sampling_window:
                corpus.append(doc[:])
                document_names.append(documents[i])
        print('\nFinal number of document_names:', len(document_names))

        # Update vocabulary
        corpus_flat = set([subitem for item in corpus for subitem in item if type(item) is list])
        vocabulary_index = set([token_to_index[token] for token in vocabulary])

        # Update vocabulary after merging sentences
        vocabulary_index = corpus_flat.intersection(vocabulary_index)
        vocabulary = [index_to_token[token_index] for token_index in vocabulary_index]
        print('Final length of vocabulary:', len(vocabulary))

        # Change names since code copied and slightly modified from action to corpus
        player_vocabulary = vocabulary
        player_vocabulary_index = vocabulary_index
        player_corpus = corpus
        player_token_to_index = token_to_index
        player_index_to_token = index_to_token

        return player_vocabulary, player_vocabulary_index, player_corpus, document_names, all_events, player_token_to_index, player_index_to_token

In [ ]:
# Train doc2vec with player corpus
def train_Doc2Vec(player_corpus) -> (Doc2Vec, KeyedVectors):
    
    # Train model
    docs = player_corpus
    docs_for_model = [TaggedDocument(doc, [i]) for i, doc in enumerate(docs)]

    model = Doc2Vec(documents=docs_for_model, min_count=1, embedding_size=32, sampling_window=1)
    model.train(docs_for_model, total_examples=len(docs_for_model), epochs=10)
        
    docs_embeddings = model.docvecs
    
    return model, docs_embeddings

In [ ]:
# Train Player2Vec model and include averaging of player matches
def player_w2v(all_events):

    # Build corpus
    print(f'\nBuilding actions corpus and applying tokenization')
    print(f'Building PlayerW2V corpus')
    player_vocabulary, player_vocabulary_index, player_corpus, document_names, all_events, player_token_to_index, player_index_to_token = players_to_corpus(all_events)
    print(document_names)
    print(player_vocabulary)

    
    print(f"Number of documents for model: {len(player_corpus)}")
    player_w2v_model, player_w2v_embeddings = train_Doc2Vec(player_corpus)

    print(f"Number of documents: {len(document_names)}")
    print(f"Number of sentences in corpus: {len(player_corpus)}")

    # Build a comfortable dict for collecting players' vectors, later to be the players_embeddings
    embedded_vocab = {}
    for i, doc in enumerate(document_names):
        embedded_vocab[doc] = player_w2v_model.infer_vector(player_corpus[i])

    embedded_vocab = pd.DataFrame.from_dict(embedded_vocab, orient='index')
    print(embedded_vocab.head(5))

    # Build players embeddings - averages of their matches
    
    print(f'\nBuilding Player embeddings')
    separator = '/'
    embedded_vocab = embedded_vocab.reset_index()
    embedded_vocab['player_name'] = embedded_vocab['index'].apply(lambda index: index.split(separator)[0])
    avg_player_embeddings = embedded_vocab.groupby('player_name').mean()

    return player_w2v_model, all_events, player_corpus, embedded_vocab, avg_player_embeddings, player_w2v_embeddings, player_vocabulary_index, player_token_to_index, player_index_to_token

In [ ]:
# Generate player vectors

player_w2v_model, all_events, player_corpus, embedded_vocab, avg_player_embeddings, player_w2v_embeddings, player_vocabulary_index, player_token_to_index, player_index_to_token = player_w2v(all_events)

In [ ]:
# Generate similarity matrix for all players

def generate_doc_similarities(model_name: str, avg_player_embeddings):
    print(f'\nCalculating Player2Vec player_similarities')
    player_similarities = {}
    for i, (ix, doc_1) in tqdm(enumerate(avg_player_embeddings.iterrows()), total=avg_player_embeddings.shape[0]):
        for j, doc_2 in avg_player_embeddings[i:].iterrows():
            cosine_similarity = np.dot(doc_1, doc_2) / (np.linalg.norm(doc_1) * np.linalg.norm(doc_2))
            player_similarities[(ix, j)] = {'cosine': cosine_similarity}
    
    return player_similarities

In [ ]:
player_similarities = generate_doc_similarities('Player2Vec', avg_player_embeddings)

In [ ]:
# Print most similar players to given player

def get_similar_players(player_name, player_similarities):
    
    # (player 1 , player 2) : similarity
    player_similarities = {(key[0].lower(), key[1].lower()): val for key, val in player_similarities.items()}

    # Show most similar players by cosine similarity.
            
    curr_player_cosine_sim = {key[1]: val['cosine'] for key, val in player_similarities.items() if key[0] == player_name}
    top_cosine_similarities = pd.Series(curr_player_cosine_sim).sort_values(ascending=False)
    top_cosine_similarities = pd.Series(top_cosine_similarities[1:12].index, name='Player name')

    return top_cosine_similarities

def print_similar_players(player_name, top_cosine_similarities):
    print(f"Top 10 similar players to {player_name}: (By Cosine Similarity)")
    print(top_cosine_similarities)

In [ ]:
# Results for 'Suarez'
top_cosine_similarities = get_similar_players('luis alberto suárez díaz', player_similarities)
print_similar_players('luis alberto suárez díaz', top_cosine_similarities)

In [ ]:
# Reduces the player vectors to two dimensions to be plotted using the UMAP library.
def get_reduced_representation(data):
    
    dimensions_reducer = umap.UMAP()
    data = dimensions_reducer.fit_transform(data)
    
    return dimensions_reducer, data

# Plot player embeddings using UMAP/Plotly
def plot_embeddings(embedded_vocab, player_metadata):
    doc_name_separator='/'

    embedded_vocab_copy = embedded_vocab.copy()
    dimensions_reducer, embedding = get_reduced_representation(embedded_vocab_copy)

    for feature in [0, 1]:
        embedded_vocab_copy[f'umap_{feature}'] = embedding[:, feature]


    # Color the plotted players by position. Add team name into custom_data and name as label/hover_name
    embedded_vocab_copy['color'] = [player_metadata.get(player.title(), {'position_name': ''})['position_name'] for player in embedded_vocab_copy.index.copy()]
    embedded_vocab_copy['custom_data'] = [player_metadata.get(player.title(), {'team_name': ''})['team_name'] for player in embedded_vocab_copy.index.copy()]

    embedded_vocab_copy['label'] = embedded_vocab_copy.index.copy()
    embedded_vocab_copy['hover_name'] = embedded_vocab_copy.index.copy()


    fig = px.scatter(embedded_vocab_copy, x='umap_0', y='umap_1', color='color',
                     hover_name='hover_name', hover_data=["custom_data", "label"],
                     title='UMAP Projections of Players by Position')
    fig.update_traces(textposition='top center')

    fig.show()
    

In [ ]:
plot_embeddings(avg_player_embeddings, player_metadata=enriched_player_metadata)